Libraties

In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import os
from glob import glob
import cv2

In [3]:
base_path = os.path.join(os.getcwd(), 'HAM10000_Images')
for dirname, _, _ in os.walk(base_path):
    print(dirname)

In [4]:
base_path = r"d:\term 4\Studio\Skin\Skin-Cancer-Detection-App\Ml_Model\data\HAM10000_Images"
print("Looking in:", base_path)

d_paths = [x[0] for x in os.walk(base_path)]
print("Directories found:", d_paths)

i_paths = []
for d_path in d_paths:
    i_paths.extend(glob(os.path.join(d_path, '*')))
print("Number of files:", len(i_paths))

Looking in: d:\term 4\Studio\Skin\Skin-Cancer-Detection-App\Ml_Model\data\HAM10000_Images
Directories found: ['d:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\akiec', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\bcc', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\bkl', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\df', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\mel', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\nv', 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\vasc']
Number of files: 10022


In [4]:
d_paths

['d:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\akiec',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\bcc',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\bkl',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\df',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\mel',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\nv',
 'd:\\term 4\\Studio\\Skin\\Skin-Cancer-Detection-App\\Ml_Model\\data\\HAM10000_Images\\vasc']

In [5]:
from wolta.visual_tools import get_extensions

get_extensions(i_paths)

{'none': 7, 'jpg': 10015}

In [6]:

i_paths = []
for subfolder in ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']:
    folder_path = os.path.join(base_path, subfolder)
    image_paths = glob(os.path.join(folder_path, '*.*'))  # Get all files
    i_paths.extend(image_paths)

# Print first few paths to verify
print("Number of images found:", len(i_paths))
if len(i_paths) > 0:
    print("First image path:", i_paths[0])
    # Try reading the first image
    img = cv2.imread(i_paths[0])
    if img is not None:
        print('width: {}\nheight: {}'.format(img.shape[1], img.shape[0]))
    else:
        print("Failed to read image")
else:
    print("No images found in the directories")

Number of images found: 10015
First image path: d:\term 4\Studio\Skin\Skin-Cancer-Detection-App\Ml_Model\data\HAM10000_Images\akiec\ISIC_0024329.jpg
width: 128
height: 128


Image Augmentation

In [7]:
def crop_center(img, crop_width, crop_height):
    """Crop the center of an image."""
    h, w, _ = img.shape
    start_x = max((w - crop_width) // 2, 0)
    start_y = max((h - crop_height) // 2, 0)
    return img[start_y:start_y + crop_height, start_x:start_x + crop_width]

# Create a directory for processed images
output_base = 'processed_images'
os.makedirs(output_base, exist_ok=True)

# Get all subdirectories in HAM10000_Images
base_path = r"d:\term 4\Studio\Skin\Skin-Cancer-Detection-App\Ml_Model\data\HAM10000_Images"
d_paths = [os.path.join(base_path, d) for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

for d_path in d_paths:
    # Get directory name from path
    dir_name = os.path.basename(d_path)
    
    # Create subdirectory for each class
    current_dir = os.path.join(output_base, dir_name)
    os.makedirs(current_dir, exist_ok=True)
    
    # Get all images in current directory
    i_paths = glob(os.path.join(d_path, '*'))
    
    for i_path in i_paths:
        # Get image name
        i_name = os.path.basename(i_path)
        
        # Read and process image
        img = cv2.imread(i_path)
        if img is not None:
            edge = min(img.shape[0], img.shape[1])
            img = crop_center(img, edge, edge)
            img = cv2.resize(img, (128, 128))
            
            # Save processed image
            output_path = os.path.join(current_dir, i_name)
            try:
                cv2.imwrite(output_path, img)
            except Exception as e:
                print(f"Error saving image: {output_path}")
                print(e)
        else:
            print(f"Failed to read image: {i_path}")


DATA PREPARATION FOR TESTING AND TRAINING

In [8]:
train_ds, test_val_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_images',
    validation_split=0.2,
    subset='both',
    seed=123,
    image_size=(128, 128),
    batch_size=16
)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.
Using 2003 files for validation.


In [9]:
test_val_ds_size = tf.data.experimental.cardinality(test_val_ds).numpy()
test_val_split_size = int(0.5 * test_val_ds_size)

validation_ds = test_val_ds.take(test_val_split_size)
test_ds = test_val_ds.skip(test_val_split_size)

In [10]:
names = train_ds.class_names
num_classes = len(names)

print(names)

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


Model

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [12]:
model = Sequential([
    layers.Input(shape=[128, 128, 3]),
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,121,767 (8.09 MB)

 Trainable params: 2,121,767 (8.09 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=100
)

Epoch 1/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 40s 75ms/step - accuracy: 0.6582 - loss: 1.0447 - val_accuracy: 0.7073 - val_loss: 0.8955
Epoch 2/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 28s 55ms/step - accuracy: 0.6794 - loss: 0.8927 - val_accuracy: 0.7063 - val_loss: 0.8336
Epoch 3/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 28s 55ms/step - accuracy: 0.7016 - loss: 0.8169 - val_accuracy: 0.7312 - val_loss: 0.8059
Epoch 4/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 27s 54ms/step - accuracy: 0.7215 - loss: 0.7630 - val_accuracy: 0.7133 - val_loss: 0.7390
Epoch 5/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.7474 - loss: 0.6769 - val_accuracy: 0.7202 - val_loss: 0.8054
Epoch 6/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.7713 - loss: 0.6216 - val_accuracy: 0.7371 - val_loss: 0.8036
Epoch 7/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 32s 63ms/step - accuracy: 0.7878 - loss: 0.5627 - val_accuracy: 0.7331 - val_loss: 0.7807
Epoch 8/100
501/501 ━━━━━━━━━━━━━━━━━━━━ 159s 317ms/step - accuracy: 0.8163 - loss:

In [16]:
model.save("data/skin_cancer_model.keras")


In [20]:
loss, acc = model.evaluate(test_ds)

print(f"Test Accuracy: {acc * 100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7249 - loss: 5.6806
Test Accuracy: 73.17%


In [18]:
# Load the trained model
model = tf.keras.models.load_model("data/skin_cancer_model.keras")

# Define class labels (ensure these match your dataset labels)
class_labels = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]

# Path to test images
test_folder = "data/test"

def preprocess_image(image_path):
    """Load and preprocess the image for model prediction"""
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Resize to match your model's input size
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_image(image_path):
    """Predict the class of the image"""
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return class_labels[predicted_class], prediction[0][predicted_class]

# Iterate through test images and predict
for image_name in os.listdir(test_folder):
    image_path = os.path.join(test_folder, image_name)
    predicted_class, confidence = predict_image(image_path)
    print(f"Image: {image_name} | Predicted: {predicted_class} | Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Image: ISIC_0024308.jpg | Predicted: mel | Confidence: 90.36
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: ISIC_0024371.jpg | Predicted: mel | Confidence: 90.25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: ISIC_0024418.jpg | Predicted: mel | Confidence: 90.02
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image: ISIC_0024457.jpg | Predicted: mel | Confidence: 90.14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image: ISIC_0024459.jpg | Predicted: mel | Confidence: 90.75
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image: ISIC_0024904.jpg | Predicted: mel | Confidence: 90.05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Image: ISIC_0024973.jpg | Predicted: mel | Confidence: 90.38
